In [ ]:
import json, glob, pickle, multiprocessing

import numpy as np
import pandas as pd

from tqdm import tqdm
from scipy import spatial
from collections import Counter
from keras.preprocessing import image
import matplotlib.pyplot as plt
from skimage.feature import hog

In [ ]:
glob.glob("resources/captions with embeddings/*shirt*")

In [ ]:
dataset_path = "nontracked/dataset/shirt/"

In [ ]:
def load_image(image_name):
    return image.load_img(dataset_path+image_name+".jpg", target_size=(224, 224))

In [ ]:
def get_cosine(a, b):
    return 1 - spatial.distance.cosine(a, b)

In [ ]:
def get_image_embedding(image):
    return data[data[0]==image][1].values[0]

In [ ]:
def get_hog(img):
    fd, hog_image = hog(image.img_to_array(load_image(img)), orientations=8, pixels_per_cell=(3, 3),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
    
    return hog_image.reshape(-1)

In [ ]:
with open('resources/captions with embeddings/cap.shirt.train.json') as f:
    train = json.load(f)
with open('resources/captions with embeddings/cap.shirt.test.json') as f:
    test = json.load(f)
with open('resources/captions with embeddings/cap.shirt.val.json') as f:
    val = json.load(f)

In [ ]:
images = glob.glob("nontracked/dataset/shirt/*")

In [ ]:
images = [i.split("/")[-1][:-4] for i in images]

In [ ]:
# p = multiprocessing.Pool(45)
# image_embeddings = p.map(get_hog, images)
# image_embeddings = np.array(image_embeddings)
# pickle.dump(image_embeddings, open("nontracked/image_embeddings_shirt", 'wb'), protocol=4)

In [ ]:
image_embeddings = pickle.load(open("nontracked/image_embeddings_shirt", 'rb'))

In [ ]:
data = pd.DataFrame([images, image_embeddings]).T

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
def get_hog_image_embedding(image):
    return data[data[0]==image][1].values[0]

In [ ]:
with open('image_splits/split.shirt.train.json') as f:
    train_split = json.load(f)
train_split = list(set(train_split).intersection(set(data[0].values)))

with open('image_splits/split.shirt.val.json') as f:
    val_split = json.load(f)
val_split = list(set(val_split).intersection(set(data[0].values)))

with open('image_splits/split.shirt.test.json') as f:
    test_split = json.load(f)
test_split = list(set(test_split).intersection(set(data[0].values)))

In [ ]:
len(train_split), len(val_split), len(test_split)

In [ ]:
# train_text = {i:[] for i in train_split}
# for i in tqdm(train):
    
#     if i['target'] in train_split:
#         train_text[i['target']].append(i['captions'][1])

#     if i['candidate'] in train_split:
#         train_text[i['candidate']].append(i['captions'][0])

In [ ]:
# Counter([len(train_text[i]) for i in train_text])

In [ ]:
# val_text = {i:[] for i in val_split}
# for i in tqdm(val):
    
#     if i['target'] in val_split:
#         val_text[i['target']].append(i['captions'][1])

#     if i['candidate'] in val_split:
#         val_text[i['candidate']].append(i['captions'][0])

In [ ]:
test_text = {i:[] for i in test_split}
for i in tqdm(test):
    
#     if i['target'] in test_split:
#         test_text[i['target']].append(i['captions'][1])

    if i['candidate'] in test_split:
        test_text[i['candidate']].append(i['captions'][0])

In [ ]:
import gensim.models as g

In [ ]:
path = "nontracked/enwiki_dbow/doc2vec.bin"

In [ ]:
doc2vec = g.Doc2Vec.load(path)

In [ ]:
# text_embeds = {}

# for i in tqdm(train_text):
#     t = train_text[i]
#     if len(t) > 0:
#         text_embeds[i] = doc2vec.infer_vector(" ".join(t).split(" ")).tolist()

In [ ]:
# val_text_embeds = {}

# for i in tqdm(val_text):
#     t = val_text[i]
#     if len(t) > 0:
#         val_text_embeds[i] = doc2vec.infer_vector(" ".join(t).split(" ")).tolist()

In [ ]:
test_text_embeds = {}

for i in tqdm(test_text):
    t = test_text[i]
    if len(t) > 0:
        test_text_embeds[i] = doc2vec.infer_vector(" ".join(t).split(" ")).tolist()

In [ ]:
len(test)

In [ ]:
v = test[8]

In [ ]:
def get_result(v):
    candidate = v['candidate']
#     target_caption = v['caption_embeds'][0]

    target_caption = doc2vec.infer_vector(" ".join(v['captions']).split(" ")).tolist()
    
    k = []

    rankings = []
    for t in test_split:
        s = 0
        s += get_cosine(get_hog_image_embedding(candidate), get_hog_image_embedding(t))

        if t in test_text_embeds:
            p = get_cosine(target_caption, test_text_embeds[t])
            rankings.append(s*0.2 + p*0.8)
            k.append(2)
        else:
            rankings.append(s/2)
            k.append(1)

    a = np.array(rankings)
    a = np.nan_to_num(a, 0)
    http://192.168.29.83:8044/user/william18026/notebooks/fashion-iq/traditional_shirt.ipynb#
    pro = np.take(test_split, a.argsort()[::-1])
    d = np.where(pro==candidate)
    pro = np.delete(pro, d)[:50].tolist()
    
    return {"candidate": v['candidate'], "ranking": pro} #, "k": np.take(k, a.argsort()[::-1])}

In [ ]:
r = get_result(v)

In [ ]:
p = multiprocessing.Pool(45)

In [ ]:
results = p.map(get_result, test)

In [ ]:
with open('shirt.predict.json', 'w') as outfile:
    json.dump(results, outfile)

In [ ]:
# np.where(r['ranking']=='B0077SM1Z0')

In [ ]:
# image_embeddings = np.load("nontracked/image_embeddings/dress.npy", allow_pickle=True)

In [ ]:
# with open('nontracked/image_embeddings/image_embeddings_order.pickle', 'rb') as f:
#     image_order = pickle.load(f)['dress']
# image_order = [i.split("/")[-1][:-4] for i in image_order]

In [ ]:
# data = pd.DataFrame([image_order, image_embeddings]).T

In [ ]:
# data.shape

In [ ]:
# train_text_has_caption = [i for i in train_text if len(train_text[i])>0]
# train_text_no_caption = [i for i in train_text if len(train_text[i])==0]
# len(train_text_no_caption), len(train_text_has_caption)

In [ ]:
# image_embeddings_has_caption = []
# for i in tqdm(train_text_has_caption):
#     image_embeddings_has_caption.append(get_embeddings(i))
# image_embeddings_has_caption = np.array(image_embeddings_has_caption)

In [ ]:
# image_embeddings_no_caption = []
# for i in tqdm(train_text_no_caption):
#     image_embeddings_no_caption.append(get_embeddings(i))
# image_embeddings_no_caption = np.array(image_embeddings_no_caption)